In [84]:
import nltk
import sklearn
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crfsuite
from sklearn_crfsuite import metrics
import os
from pathlib import Path
import re
import pickle
import random

NUM_JANELA=4

In [4]:
entidade = ['dor', 'no', 'braço', 'esquerdo','e','direito']

lista=list()
for i in range(1, len(entidade)+1):
    #lista.append(entidade[i-1])
    for j in range(len(entidade) - i + 1):
        lista.append(entidade[j:j + i])
        
lista

[['dor'],
 ['no'],
 ['braço'],
 ['esquerdo'],
 ['e'],
 ['direito'],
 ['dor', 'no'],
 ['no', 'braço'],
 ['braço', 'esquerdo'],
 ['esquerdo', 'e'],
 ['e', 'direito'],
 ['dor', 'no', 'braço'],
 ['no', 'braço', 'esquerdo'],
 ['braço', 'esquerdo', 'e'],
 ['esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo'],
 ['no', 'braço', 'esquerdo', 'e'],
 ['braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e'],
 ['no', 'braço', 'esquerdo', 'e', 'direito'],
 ['dor', 'no', 'braço', 'esquerdo', 'e', 'direito']]

In [5]:
def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

def replaceWhiteSpaces(str):
    return re.sub('\s{2,}',' ',str)

def save_obj(name, obj):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f)

def load_obj(name):
    print('Load obj em: ', 'obj/' + name + '.pkl')
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [6]:
dic_sentencesTrain = load_obj(r'../spanclassification/obj/dic_sentencesTrain')
dic_sentencesDev = load_obj(r'../spanclassification/obj/dic_sentencesDev')
dic_sentencesTest = load_obj(r'../spanclassification/obj/dic_sentencesTest')


Load obj em:  obj/../spanclassification/obj/dic_sentencesTrain.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesDev.pkl
Load obj em:  obj/../spanclassification/obj/dic_sentencesTest.pkl


In [17]:
dic_sentencesTest[1]

[[['Em', 0, 59],
  ['acompanhamento', 1, 62],
  ['no', 2, 77],
  ['ambualtorio', 3, 80],
  ['há', 4, 92],
  ['5', 5, 95],
  ['anos', 6, 97],
  ['por', 7, 102],
  ['FA', 8, 106],
  [',', 9, 108],
  ['uso', 10, 110],
  ['de', 11, 114],
  ['marevan', 12, 117],
  ['5mg', 13, 125],
  ['1', 14, 129],
  ['x', 15, 131],
  ['ao', 16, 133],
  ['dia', 17, 136],
  ['.', 18, 139]],
 [['FA', [8], 'Problema'], ['marevan 5mg', [12, 13], 'Tratamento']]]

In [29]:
def getListaPositiva(dic_sentences):
    # só positivas:
    lista=list()
    numJanela=NUM_JANELA
    for key, value in dic_sentences.items():
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            continue
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
    return lista

In [31]:
lista = getListaPositiva(dic_sentencesTest)
lista[:5]

[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'Problema'],
 ['marevan 5mg',
  ['FA', ',', 'uso', 'de'],
  ['1', 'x', 'ao', 'dia'],
  'Tratamento'],
 ['Comorbidades', [], [':', 'DM', 'há', '10'], 'Problema'],
 ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'Problema'],
 ['metformina 850mg',
  ['anos', 'em', 'uso', 'de'],
  ['3', 'cp', '/', 'dia'],
  'Tratamento']]

In [82]:
dicPostagger['cardiaco']

'ADJ'

In [98]:
dicPostagger = load_obj('../spanclassification/obj/dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

Load obj em:  obj/../spanclassification/obj/dic_postagger.pkl


'N'

In [120]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(lista, word2cluster, dicPostagger):
    features = list()
    for sent in lista:
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.append([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[-2:]=' + entidade[-2:],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    return features


def sent2features(sent, word2cluster):
    return [word2features(sent, i, word2cluster) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters(r"clusters/cluster-50.tsv")

In [121]:
print('dor:',word2cluster['dor'])
print('esquerdo:',word2cluster['esquerdo'])
print('direito:',word2cluster['direito'])
print('peito:',word2cluster['peito'])
print('no:',word2cluster['no'])
print('na:',word2cluster['na'])
print('coração:',word2cluster['coração'])
print('válvula:',word2cluster['válvula'])
print('cardíaca:',word2cluster['cardíaca'])
print('miocárdica:',word2cluster['miocárdica'])
print('mitral:',word2cluster['mitral'])
print('mmii:',word2cluster['mmii'])
print('coronária:',word2cluster['coronária'])

dor: 25
esquerdo: 11
direito: 11
peito: 31
no: 24
na: 24
coração: 11
válvula: 29
cardíaca: 9
miocárdica: 46
mitral: 41
mmii: 7
coronária: 5


In [122]:
word2f = word2features(lista, word2cluster, dicPostagger)

In [127]:
print(word2f[:40])

[['bias', 'word.lower=fa', 'word[-3:]=FA', 'word[-2:]=FA', 'word.isupper=True', 'word.istitle=False', 'word.isdigit=False', 'word.cluster=8', 'postag=N'], '-1:word.lower=há', '-1:word.istitle=False', '-1:word.isupper=False', '-1:postag=V', '-2:word.lower=5', '-2:word.istitle=False', '-2:word.isupper=False', '-2:postag=NUM', '-3:word.lower=anos', '-3:word.istitle=False', '-3:word.isupper=False', '-3:postag=N', '-4:word.lower=por', '-4:word.istitle=False', '-4:word.isupper=False', '-4:postag=PREP', '+1:word.lower=,', '+1:word.istitle=False', '+1:word.isupper=False']


In [39]:
dic_predictions_results_ner = load_obj(r'../spanclassification/obj/dic_predictions_results_ner_800')

Load obj em:  obj/../spanclassification/obj/dic_predictions_results_ner_800.pkl


In [46]:
dic_predictions_results_ner[42]

[[['Paciente', 0],
  ['em', 1],
  ['tratamento', 2],
  ['para', 3],
  ['ICC', 4],
  ['diastólica', 5],
  ['há', 6],
  ['cerca', 7],
  ['de', 8],
  ['5', 9],
  ['a', 10],
  ['com', 11],
  ['melhora', 12],
  ['importante', 13],
  ['dos', 14],
  ['sintomas', 15],
  ['após', 16],
  ['início', 17],
  ['do', 18],
  ['tratamento', 19],
  ['.', 20]],
 [['ICC diastólica', [4, 5], 'Problema'], ['tratamento', [19], 'Tratamento']]]

In [86]:
# para corpus GENIA, ideal pq pega as descontinuas
# no nestedclin, as descontinuas
def powersetGenia(s):
    x = len(s)
    masks = [1 << i for i in range(x)]
    for i in range(1 << x):
        yield [ss for mask, ss in zip(masks, s) if i & mask]
        
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))


[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [22]:
#samplecorpusformat = [[['POI', 'B-Abbreviation'],  ['DE', 'O'],  ['LAVAGEM', 'O'],  ['+', 'O'],  ['CURETA', 'O'],  ['DE', 'O'],  ['TECIDO', 'O'],  ['NECROTICO', 'O'],  ['.', 'O']], [['18', 'O'],  [':', 'O'],  ['00', 'O'],  [':', 'O'],  ['PACIENTE', 'O'],  ['RETORNOU', 'O'],  ['DO', 'O'],  ['CC', 'B-Abbreviation'],  ['LUCIDO', 'O'], [',', 'O'],  ['ORIENTADO', 'O'],  [',', 'O'],  ['COMUNICATIVO', 'O'],  [';', 'O'], ['MANTEM', 'O'],  ['AVP', 'B-Abbreviation'],  ['COM', 'O'],  ['STP', 'B-Abbreviation'], ['.', 'O']]]

# corpus:
# gravar os indices dos tokens
# para cada entidade, gerar a combinação possivel de tokens e pegar as features

